here we are going to talk about another creational design pattern which is singleton<br>

as you know we have different ways to create a singleton class


In [ ]:
# way 1 :
# classic Singleton - Singleton Allocator

class DataBase:
  _instance = None
  def __new__(cls,*args,**kwargs):
    if cls._instance is None:
      cls._instance=super(DataBase,cls).__new__(cls,*args,**kwargs)
    return cls._instance
d1=DataBase()
d2=DataBase()
print(d1==d2)


True


now what if we write a simple __init__ ?

In [ ]:
# way 1 :
# classic Singleton - Singleton Allocator

class DataBase:
  _instance = None
  def __init__(self):
    print("hi")
  def __new__(cls,*args,**kwargs):
    print("hello")
    if cls._instance is None:
      cls._instance=super(DataBase,cls).__new__(cls,*args,**kwargs)
    return cls._instance
d1=DataBase()
d2=DataBase()
print(d1==d2)


hello
hi
hello
hi
True


what we see is that after __new__ always the __init__ is called too ,although it compelete the task of having one object but it maybe not pleasent for some developers to see that __init__ is calling twice , and also if __init__ has some code that change the state of the cls._instance then it may not be singleton any longer .<br> so lets rewtie it in a way that solve this problem

In [ ]:
# way 1 :
# classic Singleton - Singleton Allocator
# ensuring __init__ is called just once

class DataBase:
  _instance = None

  def __new__(cls,*args,**kwargs):
    print("hello")
    if cls._instance is None:
      cls._instance=super(DataBase,cls).__new__(cls,*args,**kwargs)
      cls._instance._initialized = False
    return cls._instance

  def __init__(self):
    if self._initialized:
      # Put your initialization logic here
      # This will run only once
      print( "hi")
      return
    self._initialized = True
d1=DataBase()
d2=DataBase()
print(d1==d2)

hello
hello
hi
True


In [ ]:
# way 2
# singleton decorator

def singleton(cls):
  instance={}
  def get_instance(*args , **kwargs):
    if cls not in instance:
      instance[cls]=cls(*args,**kwargs)
    return instance[cls]
  return get_instance

@singleton
class DataBase:
  def __init__(self):
    print("hi")

d1=DataBase()
d2=DataBase()
print(d1==d2)


hi
True


In [ ]:
# way 3
# metaclass singleton

class Singleton(type):
  _instances={}
  def __call__(cls,*args,**kwargs):
    if cls not in cls._instances:
      cls._instances[cls]=super(Singleton,cls).__call__(*args,**kwargs)
    return cls._instances[cls]

class DataBase(metaclass=Singleton):
  def __init__(self):
    print("hi")

d1=DataBase()
d2=DataBase()
print(d1==d2)

hi
True


In [ ]:
# way 4
# Borg / Monostate singleton
# with __init__
class Borg:
  _shared_state={}
  def __init__(self):
    self.__dict__=self._shared_state


class DataBase(Borg):
  def __init__(self,**kwargs):
    super().__init__()
    print("here in the Databse init")
    print("__dict__ : ",self.__dict__)
    print("shared_state : ",self._shared_state)
    self._shared_state.update(kwargs)
    print("__dict__ : ",self.__dict__)
    print("shared_state : ",self._shared_state)

# # d1=DataBase({"hi":"hello"})
d1=DataBase(hi="hello")
d2=DataBase(a="b")
print(d1==d2)
print(d1._shared_state)
print(d2._shared_state)
d1._shared_state==d2._shared_state



here in the Databse init
__dict__ :  {}
shared_state :  {}
__dict__ :  {'hi': 'hello'}
shared_state :  {'hi': 'hello'}
here in the Databse init
__dict__ :  {'hi': 'hello'}
shared_state :  {'hi': 'hello'}
__dict__ :  {'hi': 'hello', 'a': 'b'}
shared_state :  {'hi': 'hello', 'a': 'b'}
False
{'hi': 'hello', 'a': 'b'}
{'hi': 'hello', 'a': 'b'}


True

In [ ]:
# way 4
# Borg / Monostate singleton
# with __new__

class Borg:
  _shared_state={}
  def __new__(cls,*args,**kwargs):
    instance=super(Borg,cls).__new__(cls)
    instance.___dict__=cls._shared_state
    return instance

class DataBase(Borg):
  def __init__(self,**kwargs):
    self._shared_state.update(kwargs)

# # d1=DataBase({"hi":"hello"})
d1=DataBase(hi="hello")
d2=DataBase(a="b")
print(d1==d2)
print(d1._shared_state)
print(d2._shared_state)


False
{'hi': 'hello', 'a': 'b'}
{'hi': 'hello', 'a': 'b'}


now lets have some awfull code and refacore them

In [ ]:
# awful
class DatabaseConnection:
    def __init__(self):
        self.connection = self.connect_to_database()

    def connect_to_database(self):
        return "Database connection established"

    def query(self, sql):
        return f"Executing query: {sql}"

# Creating multiple instances of DatabaseConnection
db1 = DatabaseConnection()
db2 = DatabaseConnection()
db3 = DatabaseConnection()

print(db1.query("SELECT * FROM users"))  # Output: Executing query: SELECT * FROM users
print(db2.query("SELECT * FROM products"))  # Output: Executing query: SELECT * FROM products
print(db3.query("SELECT * FROM orders"))  # Output: Executing query: SELECT * FROM orders

# Each instance has its own state
print(db1 is db2)  # Output: False
print(db2 is db3)  # Output: False


Executing query: SELECT * FROM users
Executing query: SELECT * FROM products
Executing query: SELECT * FROM orders
False
False


In [31]:
# refactored
# way 1 : allocator
print("way 1 : allocator")
class DataBaseSingletonAllocator:
  _instance=None
  def __new__(cls,*args,**kwargs):
    if cls._instance is None:
      cls._instance=super(DataBaseSingletonAllocator,cls).__new__(cls,*args,**kwargs)
    return cls._instance

  def __init__(self):
    self.connection = self.connect_to_database()

  def connect_to_database(self):
    return "Database connection established"

  def query(self, sql):
    return f"Executing query: {sql}"

db1 = DataBaseSingletonAllocator()
db2 = DataBaseSingletonAllocator()
db3 = DataBaseSingletonAllocator()

print(db1.query("SELECT * FROM users"))
print(db2.query("SELECT * FROM products"))
print(db3.query("SELECT * FROM orders"))

# Each instance has its own state
print(db1 is db2)
print(db2 is db3)


# way 2 : decorator
print("way 2 : decorator")
def SingletoneDecorator(cls):
  instance={}
  def get_instance(*args,**kwargs):
    if cls not in instance:
      instance[cls]=cls(*args,**kwargs)
    return instance[cls]
  return get_instance

@SingletoneDecorator
class DataBaseSingletoneDecorator:
  def __init__(self):
    self.connection = self.connect_to_database()

  def connect_to_database(self):
    return "Database connection established"

  def query(self, sql):
    return f"Executing query: {sql}"


db4 = DataBaseSingletoneDecorator()
db5 = DataBaseSingletoneDecorator()
db6 = DataBaseSingletoneDecorator()

print(db4.query("SELECT * FROM users"))
print(db5.query("SELECT * FROM products"))
print(db6.query("SELECT * FROM orders"))

# Each instance has its own state
print(db4 is db5)
print(db5 is db6)


# way 3 : metaclass
print("way 3 : metaclass")
class SingletoneMetaclass(type):
  _instance={}
  def __call__(cls,*args,**kwargs):
    if cls not in cls._instance:
      cls._instance[cls]=super(SingletoneMetaclass,cls).__call__(*args,**kwargs)
    return cls._instance[cls]

class DataBaseSingletonMetaclass(metaclass=SingletoneMetaclass):
    def __init__(self):
        self.connection = self.connect_to_database()

    def connect_to_database(self):
        return "Database connection established"

    def query(self, sql):
        return f"Executing query: {sql}"

db7 = DataBaseSingletonMetaclass()
db8 = DataBaseSingletonMetaclass()
db9 = DataBaseSingletonMetaclass()

print(db7.query("SELECT * FROM users"))
print(db8.query("SELECT * FROM products"))
print(db6.query("SELECT * FROM orders"))

# Each instance has its own state
print(db7 is db8)
print(db8 is db9)


# way 4 : Borg
print("way 4 : Borg")
class Borg:
  _share_state={}
  def __new__ (cls,*args,**kwargs):

    instance=super(Borg,cls).__new__(cls)
    instance.__dict__=cls._share_state
    return instance

class DataBaseSingletonBorg(Borg):
    def __init__(self):
        self.connection = self.connect_to_database()

    def connect_to_database(self):
        return "Database connection established"

    def query(self, sql):
        return f"Executing query: {sql}"

db10 = DataBaseSingletonBorg()
db11 = DataBaseSingletonBorg()
db12 = DataBaseSingletonBorg()

print(db10.query("SELECT * FROM users"))
print(db11.query("SELECT * FROM products"))
print(db12.query("SELECT * FROM orders"))

# Each instance has its own state
print(db10 is db11)
print(db11 is db12)



way 1 : allocator
Executing query: SELECT * FROM users
Executing query: SELECT * FROM products
Executing query: SELECT * FROM orders
True
True
way 2 : decorator
Executing query: SELECT * FROM users
Executing query: SELECT * FROM products
Executing query: SELECT * FROM orders
True
True
way 3 : metaclass
Executing query: SELECT * FROM users
Executing query: SELECT * FROM products
Executing query: SELECT * FROM orders
True
True
way 4 : Borg
Executing query: SELECT * FROM users
Executing query: SELECT * FROM products
Executing query: SELECT * FROM orders
False
False
